학습이라는 건 Weight / Bias 의 값을 최적, 즉 y_hat 과 y 의 차이가 최소가 되도록 값을 변환해 나가는 일련의 과정이다. 수학적으로는 이를 미분을 사용해서 찾아 나가는데 (미분의 개념 -> x에 대한 y의 변화량이므로), 통상적인 미분을 활용한 경사하강법 Gradient Decent 보다 훨씬 유리한 다양한 알고리즘들이 이미 개발되어 있다. torch 에서는 이를 그냥 가져와서 쓸 수 있으며 주로 사용되는 알고리즘은 다음과 같다.

```python
torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False, *, maximize=False)
torch.optim.RMSprop(params, lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
torch.optim.SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False, *, maximize=False)  
torch.optim.AdamW(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False, *, maximize=False, foreach=None, capturable=False, differentiable=False, fused=None)  
torch.optim.RAdam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, *, foreach=None, differentiable=False)
```

일반적으로는 기본값을 그대로 사용해도 큰 문제가 없다. 물론 학습기울기(learning rate) 등을 스케쥴러에 맞추어 손보면 더 빠르게 학습을 할 수 있지만, 통상 이런건 마이너튜닝에 가깝기 때문에 여기에 너무 큰 에너지와 시간을 쏟을 필요가 없다는게 최근 학계의 흐름이다.

물론 옵티마이져도 커스텀으로 만들어 쓸 수 있다. 다음의 코드와 같이 모듈에서 configure_optimizer method 를 변경해주면 된다.

In [2]:
from torch.optim import Adam
import torch.optim.lr_scheduler as lr_scheduler


In [ ]:

## no learning rate change
def configure_optimizer(self):
    return Adam(self.parameters(), lr=1e-3)


In [ ]:
## multiple optimizer (for example : GAN)
def configure_optimizer(self):
    gen_opt = Adam(self.model_gen.parameters(), lr=0.01)
    dis_opt = Adam(self.model_dis.parameters(), lr=0.02 )
    
    return gen_opt, dis_opt 

In [ ]:
## multiple optimizer + own scheduler 

def configure_optimizer(self):
    gen_opt = Adam(self.model_gen.parameters(), lr = 0.01)
    dis_opt = Adam(self.model_dis.parameters(), lr = 0.02)
    gen_sch = {
        'scheduler' : lr_scheduler.ExponentialLR(gen_opt, 0.99),
        'lnterval' : 'step' 
    } ## called after each traning step or epoch.

    dis_sch = lr_scheduler.ExponentialLR(dis_opt, 0.99) #called after every epoch

    return [gen_opt, dis_opt], [gen_sch, dis_sch]




마지막예에서는 스케쥴링을 보여줬는데, 기본적으로 초반에는 빠르게, 뒤에서는 파인튜닝하는게 목적이므로, 통상 epoch 이 진행됨에 따라 lr 을 바꿔주며 이에 대해 미리 스케쥴링이 가능하다. 보다 자세한 사용법은 아래와 같다. 먼저 다음과 같이 옵티마이저를 만들고, 그 아래에 다음의 코드를 쓸 수 있다.

예를들어 학습 횟수(epoch)마다 1/10 씩 감소시키고 싶다면 stepLR 을 쓰면 된다. 

In [ ]:

def configure_optimizers(self):
    optimizer = Adam(self.model_gen.parameters(), lr=0.05)  ## 초기에 0.05로 시작한다. 기본값인 0.01 보다 큰 값.
    
    # Step-wise하게 바꿀 때 (딱딱 떨어지게 줄여나감)
    lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    # lr = 0.05   epoch < 30
    # lr = 0.005   30 ~ 60 
    # lr = 0.0005  60 ~ 90 
    # ... 
    # 
    return [optimizer], [lr_scheduler]

또는 등간격이 아니라, 사용자 임의의 간격에 따라 (30회, 90회 등등) 1/10씩 감소시키고 싶다면 MultiStepLR 을 쓰면 된다. 

In [ ]:



def configure_optimizers(self):
    optimizer = Adam(self.model_gen.parameters(), lr=0.05)  ## 초기에 0.05로 시작한다. 기본값인 0.01 보다 큰 값.    
    # Multi-step LR, Milestone 을 지정하여 step down (마일스톤에 도달할 때마다 gamma 배로 곱해짐)
    lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30, 90], gamma=0.1)
    # lr = 0.05   epochs < 30
    # lr = 0.005  30 ~ 90
    # lr = 0.0005 90 ~ 

   return [optimizer], [lr_scheduler]



또는 함수를 사용할 수 있다. lambda 함수를 사용해서 아래와 같이 함수를 지정해주거나, 


In [ ]:

def configure_optimizers(self):
    optimizer = Adam(self.model_gen.parameters(), lr=0.05)  ## 초기에 0.05로 시작한다. 기본값인 0.01 보다 큰 값.    
    
    lambda1 = lambda epoch : 0.95 ** epoch 
    lr_scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda= lambda1)
    ## lr = lr_start * function (fucntion 값이 lr 이 아니라 lr 에 곱해짐)
    ## 0.05     epoch 1
    ## 0.0475   epoch 2 
    ## 0.045125 epoch 3
    ## ...
    return [optimizer], [lr_scheduler]

또는 단순 함수보다 복잡한 경우라면 사용자 지정 함수를 정의하여

In [ ]:

def configure_optimizers(self):
    optimizer = Adam(self.model_gen.parameters(), lr=0.05)  ## 초기에 0.05로 시작한다. 기본값인 0.01 보다 큰 값.

    def myfunc(epoch):
        return 0.5 ** (epoch // 10)
    
    lr_scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda= myfunc)
    ## 0.05 1~9
    ## 0.025 10~19 
    ## 0.0125 20~29
    return [optimizer], [lr_scheduler]


지정할 수 도 있다. 또한 exponential 을 활용하여 decay하는 경우 아래와 같이 적용 가능하다.

In [ ]:

def configure_optimizers(self):
    optimizer = Adam(self.model_gen.parameters(), lr=0.05)  ## 초기에 0.05로 시작한다. 기본값인 0.01 보다 큰 값.

    gamma = 0.98
    lr_scheduler = lr_scheduler.ExponentialLR(optimizer, gamma, last_epoch= -1, verbose= False)

    return [optimizer], [lr_scheduler]

마지막으로 이렇게 사용할 일은 거의 없긴 하겠지만, 모델 내에서 레이어별로 Learning rate을 각기 따로 주고 싶을 때에는 아래와 같이 활용이 가능하다. 

In [ ]:

## set start learning rate
lr_group1 = 0.5
lr_group2 = 0.1 

# define the parameter groups
param_group1 = [p for p in model.layer1.parameters()]
param_group2 = [p for p in model.layer2.parameters()]

# create optimizer 
optimizer = optim.SGD([
    {'params' : param_group1, 'lr' : lr_group1},
    {'params' : param_group2, 'lr' : lr_group2}
])

lambda1 = lambda epoch : epoch // 30 
lambda2 = lambda epoch : 0.95 ** epoch 

lr_scheduler = lr_scheduler.LambdaLR( optimizer, lr_lambda=[lambda1, lambda2])

